In [23]:
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt

In [1]:
"""
config_map = {
    "model": config.ModelConfig(),
    "diffuser": config.DiffusionConfig(),
    "preprocess": config.PreprocessConfig(),
    "seq_diffuser": config.SeqDiffusionConfig(),
    
}
IGNORE_KEYS = ["d_time_emb", "d_time_emb_proj", "use_motif_timestep", "prob_self_cond", "str_self_cond"]
def compare_recurse(dict1, dict2, prefix=""):
    for k, v in dict1.items():
        if k in IGNORE_KEYS:
            continue
        if k not in dict2:
            print(f"Missing {prefix} {k}={v} in dict2")
        elif isinstance(v, dict):
            compare_recurse(v, dict2[k], prefix=f"{prefix}_{k}")
        elif v != dict2[k]:
            print(f"Mismatch {prefix} {k}={v} to {dict2[k]} (default)")
    return      

conf_dict = ckpt["config_dict"]
key_diff = set(conf_dict.keys()).difference(set(config_map.keys()))
if any(key_diff):
    print("Missing def for ", key_diff)
    
for k, dataclass_conf in config_map.items():
    if k in conf_dict:
        compare_recurse(conf_dict[k], asdict(dataclass_conf), prefix=k)
"""
print("ignore")

ignore


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import py3Dmol
import torch
import numpy as np
from tqdm import tqdm

from proteome import protein
from proteome.models.design.rfdiffusion import config
from proteome.models.design.rfdiffusion.modeling import RFDiffusionForUnconditionalStructureDesign, RFDiffusionForScaffoldGuidedBinderDesign
from proteome.models.design.rfdiffusion.samplers import SelfConditioning, ScaffoldedSampler
from proteome.models.design.rfdiffusion.util import writepdb

PyRosetta-4 2023 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python310.Release 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python310.Release r353 2023.27+release.e3ce6ea e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=596941989 seed_offset=0 real_seed=596941989 thread_index=0
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=596941989 RG_type=mt19937


In [4]:
pdb = "1qys.pdb"
with open(pdb, mode="r") as f:
    pdb_str = f.read()
    
input_structure = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [5]:
with open('insulin_target.pdb', mode="r") as f:
    target_pdb_str = f.read()

with open('5L33.pdb', mode="r") as f:
    scaffold_pdb_str = f.read()

In [71]:
def run_sampler(sampler, x_init, seq_init):
    denoised_xyz_stack = []
    px0_xyz_stack = []
    seq_stack = []
    plddt_stack = []
    
    x_t = torch.clone(x_init)
    seq_t = torch.clone(seq_init)
    # Loop over number of reverse diffusion time steps.
    for t in tqdm(range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1)):
        px0, x_t, seq_t, plddt = sampler.sample_step(
            t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
        )
        px0_xyz_stack.append(px0)
        denoised_xyz_stack.append(x_t)
        seq_stack.append(seq_t)
        plddt_stack.append(plddt[0])

    denoised_xyz_stack = torch.stack(denoised_xyz_stack)
    px0_xyz_stack = torch.stack(px0_xyz_stack)
    plddt_stack = torch.stack(plddt_stack)

    # Save outputs
    final_seq = seq_stack[-1]
    
    # Output glycines, except for motif region
    final_seq = torch.where(
        torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
    )  # 7 is glycine
    
    bfacts = torch.ones_like(final_seq.squeeze())
    # make bfact=0 for diffused coordinates
    bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0

    return protein.Protein(
        atom_positions=denoised_xyz_stack[-1].numpy(),
        aatype=final_seq.numpy(),
        atom_mask=np.ones(denoised_xyz_stack[-1].shape[:2], dtype=np.bool_),
        residue_index=np.arange(1, len(final_seq) + 1, dtype=np.int32),
        b_factors=bfacts[:, None].numpy().repeat(14, axis=-1),
        chain_index=np.array([protein.PDB_CHAIN_IDS.index(char) for char in sampler.chain_idx]),
    )

In [24]:
def show_pdb_string(pdb_str):
    PLDDT_BANDS = [
      (0, 50, '#FF7D45'),
      (50, 70, '#FFDB13'),
      (70, 90, '#65CBF3'),
      (90, 100, '#0053D6')
    ]
    view = py3Dmol.view(width=800, height=600)
    view.addModelsAsFrames(pdb_str)
    
    color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
    style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}
    
    style['stick'] = {}
    
    view.setStyle({'model': -1}, style)
    view.zoomTo()
    return view

inference.symmetry="C6" 
inference.num_designs=10 
inference.output_prefix="example_outputs/C6_oligo" 
'potentials.guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.1"]' 
potentials.olig_intra_all=True 
potentials.olig_inter_all=True 
potentials.guide_scale=2.0 
potentials.guide_decay="quadratic" 
'contigmap.contigs=[480-480]'

## Design cyclic oligos

```bash
../scripts/run_inference.py --config-name=symmetry 
inference.symmetry="C6" 
inference.num_designs=10 
inference.output_prefix="example_outputs/C6_oligo" 
'potentials.guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.1"]' 
potentials.olig_intra_all=True 
potentials.olig_inter_all=True 
potentials.guide_scale=2.0 
potentials.guide_decay="quadratic" 
'contigmap.contigs=[480-480]'
```

In [7]:
designer = RFDiffusionForUnconditionalStructureDesign(model_name="base")

In [8]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["90-90"]),
    symmetry_conf=config.SymmetryParams(symmetry="C6"),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(
        guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.1"], 
        olig_intra_all=True, 
        olig_inter_all=True, 
        guide_scale=2.0, 
        guide_decay="quadratic",
    ),
)

Successful diffuser __init__


In [9]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [11]:
designed_structure = run_sampler(sampler, x_init, seq_init)

  0%|                                                               | 0/50 [00:00<?, ?it/s]/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
100%|██████████████████████████████████████████████████████| 50/50 [00:42<00:00,  1.17it/s]


In [12]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [25]:
show_pdb_string(designed_pdb_str)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Dihedral oligos

```bash
python ../scripts/run_inference.py 
--config-name=symmetry 
inference.symmetry="D2" 
inference.num_designs=10 
inference.output_prefix="example_outputs/D2_oligo" 
'potentials.guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.1"]' 
potentials.olig_intra_all=True 
potentials.olig_inter_all=True 
potentials.guide_scale=2.0 
potentials.guide_decay="quadratic" 
'contigmap.contigs=[320-320]'
```

In [14]:
designer = RFDiffusionForUnconditionalStructureDesign(model_name="base")

In [17]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["120-120"]),
    symmetry_conf=config.SymmetryParams(symmetry="D2"),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(
        guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.1"], 
        olig_intra_all=True, 
        olig_inter_all=True, 
        guide_scale=2.0, 
        guide_decay="quadratic",
    ),
)

Successful diffuser __init__


In [18]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [19]:
designed_structure = run_sampler(sampler, x_init, seq_init)

100%|██████████████████████████████████████████████████████| 50/50 [00:41<00:00,  1.20it/s]


In [20]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [26]:
show_pdb_string(designed_pdb_str)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Enzyme

```bash
../scripts/run_inference.py 
inference.output_prefix=example_outputs/design_enzyme 
inference.input_pdb=input_pdbs/5an7.pdb 
'contigmap.contigs=[10-100/A1083-1083/10-100/A1051-1051/10-100/A1180-1180/10-100]' 
potentials.guide_scale=1 
'potentials.guiding_potentials=["type:substrate_contacts,s:1,r_0:8,rep_r_0:5.0,rep_s:2,rep_r_min:1"]' 
potentials.substrate=LLK 
inference.ckpt_override_path=../models/ActiveSite_ckpt.pt
```

In [30]:
designer = RFDiffusionForUnconditionalStructureDesign(model_name="active_site")

Downloading: "http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt" to /home/conradry71/.cache/torch/hub/checkpoints/active_site.pt
100%|███████████████████████████████████████████████████| 461M/461M [00:07<00:00, 64.6MB/s]


In [29]:
with open("5an7.pdb", mode="r") as f:
    pdb_str = f.read()
    
input_structure = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [42]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["10-100/A1083-1083/10-100/A1051-1051/10-100/A1180-1180/10-100"]),
    symmetry_conf=config.SymmetryParams(),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(
        guiding_potentials=["type:substrate_contacts,s:1,r_0:8,rep_r_0:5.0,rep_s:2,rep_r_min:1"], 
        guide_scale=1, 
        guide_decay="quadratic",
        substrate="LLK",
    ),
)

Successful diffuser __init__


In [43]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [44]:
designed_structure = run_sampler(sampler, x_init, seq_init)

  2%|█                                                      | 1/50 [00:02<02:17,  2.81s/it]

types torch.float64 torch.float64 torch.float64


  4%|██▏                                                    | 2/50 [00:05<02:10,  2.71s/it]

types torch.float64 torch.float64 torch.float64


  6%|███▎                                                   | 3/50 [00:08<02:06,  2.70s/it]

...

 96%|███████████████████████████████████████████████████▊  | 48/50 [02:07<00:05,  2.64s/it]

types torch.float64 torch.float64 torch.float64


 98%|████████████████████████████████████████████████████▉ | 49/50 [02:09<00:02,  2.64s/it]

types torch.float64 torch.float64 torch.float64


100%|██████████████████████████████████████████████████████| 50/50 [02:12<00:00,  2.65s/it]


In [45]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [46]:
show_pdb_string(designed_pdb_str)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Motifscaffolding InpaintSeq

```bash
../scripts/run_inference.py 
inference.output_prefix=example_outputs/design_motifscaffolding_inpaintseq 
inference.input_pdb=input_pdbs/5TPN.pdb 'contigmap.contigs=[10-40/A163-181/10-40]' 
inference.num_designs=10 
'contigmap.inpaint_seq=[A163-168/A170-171/A179]'
```

In [64]:
designer = RFDiffusionForUnconditionalStructureDesign(model_name="inpaint_seq")

In [65]:
with open("5tpn.pdb", mode="r") as f:
    pdb_str = f.read()
    
input_structure = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [66]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["10-40/A163-181/10-40"], inpaint_seq=["A163-168/A170-171/A179"]),
    symmetry_conf=config.SymmetryParams(),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(),
)

Successful diffuser __init__


In [67]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [68]:
designed_structure = run_sampler(sampler, x_init, seq_init)

  0%|                                                               | 0/50 [00:00<?, ?it/s]

  2%|█                                                      | 1/50 [00:00<00:33,  1.48it/s]

  4%|██▏                                                    | 2/50 [00:01<00:31,  1.55it/s]

  6%|███▎                                                   | 3/50 [00:01<00:29,  1.58it/s]

  8%|████▍                                                  | 4/50 [00:02<00:29,  1.57it/s]

...

 94%|██████████████████████████████████████████████████▊   | 47/50 [00:29<00:01,  1.59it/s]

 96%|███████████████████████████████████████████████████▊  | 48/50 [00:30<00:01,  1.61it/s]

 98%|████████████████████████████████████████████████████▉ | 49/50 [00:31<00:00,  1.60it/s]

100%|██████████████████████████████████████████████████████| 50/50 [00:31<00:00,  1.57it/s]

Protein shape torch.Size([73, 14, 3])
mask shape (73, 14)
bfacts shape (73, 14)


In [69]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [70]:
show_pdb_string(designed_pdb_str)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## MotifScaffoling with Target

```bash
python ../scripts/run_inference.py 
inference.output_prefix=example_outputs/design_motifscaffolding_with_target 
inference.input_pdb=input_pdbs/1YCR.pdb 
'contigmap.contigs=[A25-109/0 0-70/B17-29/0-70]' 
contigmap.length=70-120 
inference.num_designs=10 
inference.ckpt_override_path=../models/Complex_base_ckpt.pt
```

In [73]:
designer = RFDiffusionForUnconditionalStructureDesign(model_name="complex_base")

In [76]:
with open('1ycr.pdb', mode="r") as f:
    pdb_str = f.read()

input_structure = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [77]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["A25-109/0 0-70/B17-29/0-70"], length="70-120"),
    symmetry_conf=config.SymmetryParams(),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(),
)

Successful diffuser __init__


In [78]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [79]:
designed_structure = run_sampler(sampler, x_init, seq_init)

  0%|                                                               | 0/50 [00:00<?, ?it/s]

  2%|█                                                      | 1/50 [00:01<01:16,  1.55s/it]

  4%|██▏                                                    | 2/50 [00:03<01:12,  1.51s/it]

  6%|███▎                                                   | 3/50 [00:04<01:10,  1.49s/it]

  8%|████▍                                                  | 4/50 [00:05<01:08,  1.48s/it]

...

 92%|█████████████████████████████████████████████████▋    | 46/50 [01:08<00:05,  1.46s/it]

 94%|██████████████████████████████████████████████████▊   | 47/50 [01:09<00:04,  1.45s/it]

 96%|███████████████████████████████████████████████████▊  | 48/50 [01:11<00:02,  1.45s/it]

 98%|████████████████████████████████████████████████████▉ | 49/50 [01:12<00:01,  1.45s/it]

100%|██████████████████████████████████████████████████████| 50/50 [01:13<00:00,  1.48s/it]


In [80]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [81]:
show_pdb_string(designed_pdb_str)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Other

In [10]:
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=designer.cfg.diffuser,
    preprocess_conf=designer.cfg.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["480-480"]),
    symmetry_conf=config.SymmetryParams(symmetry="C6"),
    ppi_conf=config.PPIParams(),
    denoiser_conf=config.DenoiserParams(),
    potential_conf=config.PotentialsParams(olig_intra_all=True, olig_inter_all=True, guide_scale=2.0, guide_decay="quadratic")
    #scaffold_conf=config.ScaffoldGuidedParams(scaffoldguided=True, target_pdb_str=target_pdb_str, scaffold_pdb_str_list=[scaffold_pdb_str], target_ss=True, target_adj=True),
)

Successful diffuser __init__


TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
x_init, seq_init = sampler.sample_init()

In [9]:
denoised_xyz_stack = []
px0_xyz_stack = []
seq_stack = []
plddt_stack = []

x_t = torch.clone(x_init)
seq_t = torch.clone(seq_init)
# Loop over number of reverse diffusion time steps.
for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
    px0, x_t, seq_t, plddt = sampler.sample_step(
        t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
    )
    px0_xyz_stack.append(px0)
    denoised_xyz_stack.append(x_t)
    seq_stack.append(seq_t)
    plddt_stack.append(plddt[0])

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [12]:
# Flip order for better visualization in pymol
denoised_xyz_stack = torch.stack(denoised_xyz_stack)
denoised_xyz_stack = torch.flip(
    denoised_xyz_stack,
    [
        0,
    ],
)
px0_xyz_stack = torch.stack(px0_xyz_stack)
px0_xyz_stack = torch.flip(
    px0_xyz_stack,
    [
        0,
    ],
)

# For logging -- don't flip
plddt_stack = torch.stack(plddt_stack)

In [13]:
# Save outputs
final_seq = seq_stack[-1]

# Output glycines, except for motif region
final_seq = torch.where(
    torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
)  # 7 is glycine

bfacts = torch.ones_like(final_seq.squeeze())
# make bfact=0 for diffused coordinates
bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0

In [14]:
out = f"test.pdb"
# Now don't output sidechains
writepdb(
    out,
    denoised_xyz_stack[0, :, :4],
    final_seq,
    sampler.binderlen,
    chain_idx=sampler.chain_idx,
    bfacts=bfacts,
)

In [23]:
with open("test.pdb", mode="r") as f:
    designed_pdb = f.read()

In [24]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(designed_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [154]:
designer = RFDiffusionForUnconditionalStructureDesign()
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=config.BaseConfig.diffuser,
    preprocess_conf=config.BaseConfig.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["64-64"]),
    symmetry_conf=config.SymmetryParams(symmetry=None),
)

Successful diffuser __init__


In [155]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [156]:
denoised_xyz_stack = []
px0_xyz_stack = []
seq_stack = []
plddt_stack = []

x_t = torch.clone(x_init)
seq_t = torch.clone(seq_init)
# Loop over number of reverse diffusion time steps.
for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
    px0, x_t, seq_t, plddt = sampler.sample_step(
        t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
    )
    px0_xyz_stack.append(px0)
    denoised_xyz_stack.append(x_t)
    seq_stack.append(seq_t)
    plddt_stack.append(plddt[0])

In [157]:
# Flip order for better visualization in pymol
denoised_xyz_stack = torch.stack(denoised_xyz_stack)
denoised_xyz_stack = torch.flip(
    denoised_xyz_stack,
    [
        0,
    ],
)
px0_xyz_stack = torch.stack(px0_xyz_stack)
px0_xyz_stack = torch.flip(
    px0_xyz_stack,
    [
        0,
    ],
)

# For logging -- don't flip
plddt_stack = torch.stack(plddt_stack)

In [158]:
# Save outputs
final_seq = seq_stack[-1]

# Output glycines, except for motif region
final_seq = torch.where(
    torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
)  # 7 is glycine

bfacts = torch.ones_like(final_seq.squeeze())
# make bfact=0 for diffused coordinates
bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0

In [159]:
out = f"test.pdb"
# Now don't output sidechains
writepdb(
    out,
    denoised_xyz_stack[0, :, :4],
    final_seq,
    sampler.binderlen,
    chain_idx=sampler.chain_idx,
    bfacts=bfacts,
)

In [166]:
with open("test.pdb", mode="r") as f:
    designed_pdb = f.read()

In [165]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(designed_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Folding

In [139]:
from proteome import protein
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [140]:
sequence_designer = ProteinMPNNForSequenceDesign("vanilla_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [143]:
target_structure = protein.from_pdb_string(designed_pdb, chain_id="A")
target_structure = protein.to_backbone_only_protein(target_structure)

In [144]:
sequence, score = sequence_designer.design_sequence(target_structure)
print(f"Sequence: {sequence} with global_score {score}")

Sequence: AALAAAAAALAAAHAAAHAAAAAALAALAAADPAAAAALGAAAAAAHAAAGAAHAAAAALAAAA with global_score 0.8552732467651367


In [151]:
predicted_protein, confidence = folder.fold(sequence)
folded_pdb = protein.to_pdb(predicted_protein)

In [152]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(folded_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol